In [2]:
import os
import tensorflow as tf
from keras.layers import Rescaling  # Camada de escalonamento
from keras.optimizers import Adam, SGD  # optimizadores 
# Classe utilizada para acompanhamento durante o treinamento onde definimos os atributos que serão considerados para avaliação
from keras.callbacks import ModelCheckpoint
from tensorflow.data import AUTOTUNE
# Função que carrega o dataset de um diretório
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from keras.models import Sequential
from keras.layers import Activation, Flatten, Dense, Dropout
import matplotlib.pyplot as plt


import pandas as pd

In [14]:
base_model = EfficientNetB0(include_top=False, input_shape=(48, 48, 3))
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(7, activation="softmax"))

for layer in base_model.layers:
    layer.trainable = False
#model.load_weights('all_layers_freezed.h5')

for layer in base_model.layers[int(len(base_model.layers)/2):]: ## treinando somente a segunda metade
    layer.trainable = True

In [16]:
for l in model.layers:
    print(l.name, l.trainable)

efficientnetb0 True
flatten_3 True
dense_3 True


In [17]:
for l in base_model.layers:
    print(l.name, l.trainable)

input_4 False
rescaling_3 False
normalization_3 False
tf.math.truediv_3 False
stem_conv_pad False
stem_conv False
stem_bn False
stem_activation False
block1a_dwconv False
block1a_bn False
block1a_activation False
block1a_se_squeeze False
block1a_se_reshape False
block1a_se_reduce False
block1a_se_expand False
block1a_se_excite False
block1a_project_conv False
block1a_project_bn False
block2a_expand_conv False
block2a_expand_bn False
block2a_expand_activation False
block2a_dwconv_pad False
block2a_dwconv False
block2a_bn False
block2a_activation False
block2a_se_squeeze False
block2a_se_reshape False
block2a_se_reduce False
block2a_se_expand False
block2a_se_excite False
block2a_project_conv False
block2a_project_bn False
block2b_expand_conv False
block2b_expand_bn False
block2b_expand_activation False
block2b_dwconv False
block2b_bn False
block2b_activation False
block2b_se_squeeze False
block2b_se_reshape False
block2b_se_reduce False
block2b_se_expand False
block2b_se_excite False
bl

In [18]:
train_path = "./../data/fer2013/train/"
validation_path = "./../data/fer2013/test/"
models_path = "./models"

width = 48  
height = 48  
depth = 1 

epochs = 25
init_lr = 1e-3
batch_size = 32

color_mode = "rgb"

input_shape = (height, width, depth)
save_model = os.path.join(
        models_path, "resnet-{epoch:02d}-{accuracy:.3f}-{val_accuracy:.3f}.model")

In [21]:
train_ds = image_dataset_from_directory(
        train_path,
        seed=123,
        label_mode='categorical',
        validation_split=0.3,
        subset="training",
        color_mode=color_mode,
        image_size=(height, width),
        batch_size=batch_size
    )

val_ds = image_dataset_from_directory(
    validation_path,
    seed=123,
    label_mode='categorical',
    validation_split=0.3,
    subset="validation",
    color_mode=color_mode,
    image_size=(height, width),
    batch_size=batch_size
) 

Found 28709 files belonging to 7 classes.
Using 20097 files for training.
Found 7178 files belonging to 7 classes.
Using 2153 files for validation.


In [22]:
rescaling_layer = Rescaling(1./255) ## normalizando os valores dos pixels
# pré-busca em buffer para que se possa produzir dados do disco sem que a E/S se torne um bloqueio
train_ds = train_ds.map(lambda x, y: (rescaling_layer(x), y), 
                        num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (rescaling_layer(x), y),
                    num_parallel_calls=AUTOTUNE)


In [23]:
####model = utils.create_model(input_shape=input_shape)

In [30]:
opt = keras.optimizers.SGD(lr=init_lr, decay=init_lr / epochs)

model.compile(loss="categorical_crossentropy",
                  optimizer=opt, metrics=["accuracy", Precision()])

/home/elisaveloso/.local/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [31]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 2, 2, 1280)       4049571   
                                                                 
 flatten_3 (Flatten)         (None, 5120)              0         
                                                                 
 dense_3 (Dense)             (None, 7)                 35847     
                                                                 
Total params: 4,085,418
Trainable params: 3,734,735
Non-trainable params: 350,683
_________________________________________________________________


In [32]:
checkpoint1 = ModelCheckpoint(
    save_model, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
checkpoint2 = ModelCheckpoint(
    save_model, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint1, checkpoint2]

In [33]:
H = model.fit(train_ds,
                  validation_data=val_ds,
                  epochs=epochs,
                  verbose=1,
                  callbacks=callbacks_list
                  )



Epoch 1/25
629/629 [==============================] - ETA: 0s - loss: 1.9636 - accuracy: 0.2136 - precision: 0.2381
Epoch 1: val_loss improved from inf to 2.03414, saving model to ./models/resnet-01-0.214-0.151.model


INFO:tensorflow:Assets written to: ./models/resnet-01-0.214-0.151.model/assets


INFO:tensorflow:Assets written to: ./models/resnet-01-0.214-0.151.model/assets



Epoch 1: val_accuracy improved from -inf to 0.15095, saving model to ./models/resnet-01-0.214-0.151.model


INFO:tensorflow:Assets written to: ./models/resnet-01-0.214-0.151.model/assets


...

INFO:tensorflow:Assets written to: ./models/resnet-14-0.317-0.278.model/assets



Epoch 14: val_accuracy did not improve from 0.30144
629/629 [==============================] - 129s 205ms/step - loss: 1.7044 - accuracy: 0.3174 - precision: 0.5648 - val_loss: 1.7568 - val_accuracy: 0.2778 - val_precision: 0.6222
Epoch 15/25
629/629 [==============================] - ETA: 0s - loss: 1.6950 - accuracy: 0.3218 - precision: 0.5706
Epoch 15: val_loss did not improve from 1.75681

Epoch 15: val_accuracy did not improve from 0.30144
629/629 [==============================] - 112s 177ms/step - loss: 1.6950 - accuracy: 0.3218 - precision: 0.5706 - val_loss: 1.7750 - val_accuracy: 0.2717 - val_precision: 0.6220
Epoch 16/25
629/629 [==============================] - ETA: 0s - loss: 1.6869 - accuracy: 0.3249 - precision: 0.5859
Epoch 16: val_loss improved from 1.75681 to 1.74860, saving model to ./models/resnet-16-0.325-0.301.model


INFO:tensorflow:Assets written to: ./models/resnet-16-0.325-0.301.model/assets


INFO:tensorflow:Assets written to: ./models/resnet-16-0.325-0.301.model/assets



Epoch 16: val_accuracy did not improve from 0.30144
629/629 [==============================] - 147s 234ms/step - loss: 1.6869 - accuracy: 0.3249 - precision: 0.5859 - val_loss: 1.7486 - val_accuracy: 0.3010 - val_precision: 0.5882
Epoch 17/25
629/629 [==============================] - ETA: 0s - loss: 1.6876 - accuracy: 0.3248 - precision: 0.5944
Epoch 17: val_loss did not improve from 1.74860

Epoch 17: val_accuracy did not improve from 0.30144
629/629 [==============================] - 120s 191ms/step - loss: 1.6876 - accuracy: 0.3248 - precision: 0.5944 - val_loss: 1.7804 - val_accuracy: 0.2917 - val_precision: 0.5549
Epoch 18/25
629/629 [==============================] - ETA: 0s - loss: 1.6808 - accuracy: 0.3269 - precision: 0.5965
Epoch 18: val_loss did not improve from 1.74860

Epoch 18: val_accuracy did not improve from 0.30144
629/629 [==============================] - 116s 184ms/step - loss: 1.6808 - accuracy: 0.3269 - precision: 0.5965 - val_loss: 1.7948 - val_accuracy: 0.252

In [34]:
report = H.history

In [35]:
report = pd.DataFrame(report)

In [ ]:
report

In [36]:
report.to_csv("report-9.csv")

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(H.history['loss'], label='Training Loss')
plt.plot(H.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
val_predictions = model.predict(val_ds)
val_true_labels = []
for images, labels in val_ds:
    val_true_labels.extend(tf.argmax(labels, axis=1).numpy())

val_pred_labels = tf.argmax(val_predictions, axis=1).numpy()


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

cm = confusion_matrix(val_true_labels, val_pred_labels)
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()